# 14_한국어 전처리 패키지(Text Preprocessing Tools for Korean Text)
이 문서에서는 유용한 한국어 전처리 패키지를 다룬다. 앞서 소개한 형태소와 문장 토크나이징 도구들인 KoNLPy와 KSS(Korean Sentence Splitter)와 함께 유용하게 사용할 수 있는 패키지들이다. 

## 1. PyKoSpacing
PyKoSpacing은 한국어 띄어쓰기 패키지로 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지이다. PyKoSpacing은 대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥 러닝 모델로 준수한 성능을 가지고 있다.

코드 다운로드가 제대로 실행되지 않는 문제점이 존재한다. 

In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

## 2. Py-Hanspell
Py-Hanspell은 네이버 한글 맞춤법 검사기를 바탕으로 만들어진 패키지이다.

In [8]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [3]:
from hanspell import spell_checker

sentence = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sentence)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


위의 예시에서 알 수 있듯이, 이 패키지는 띄어쓰기도 보정해준다. 다른 예제를 시험해보자. 

In [5]:
sentence2 = '김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.'
spelled_sent = spell_checker.check(sentence2)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.


## 3. SOYNLP를 이용한 단어 토큰화
soynlp는 품사 태깅, 단어 토큰화 등을 지원하는 토크나이저이다. 비지도학습으로 단어 토큰화를 한다는 특징을 갖고 있으며, 데이터에 자주 등장하는 단어들을 분석합니다. soynlp 단어 토크나이저는 내부적으로 단어 점수 표가 있고, 그 표를 기반으로 동작한다.이 점수는 응집 확률(cohension probability)과 브랜칭 엔트로피(branching entropy)를 활용한다.

In [7]:
# !pip install soynlp

## 1. 신조어 문제
soynlp를 소개하기 전에 기존의 형태소 분석기가 가진 문제는 무엇이었는지, SOYNLP가 어떤 점에서 유용한지 정리해보자. 기존의 형태소 분석기는 신조어나 형태소 분석기에 등록되지 않은 단어 같은 경우 제대로 구분하지 못하는 단점이 있었다. 

In [9]:
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정"))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']


에이비식스는 아이돌의 이름이고, 이대휘는 에이비식스의 멤버이며, 최애돌은 최고로 애정하는 아이돌이라는 뜻이지만 위의 형태소 분석 결과에서는 단어가 죄대 따로따로 분리되어 있는 것을 볼 수 있다.<br>

그렇다면 텍스트 데이터에서 특정 문자 시퀀스가 자주 함께 등장하는 빈도가 높고, 앞뒤로 조사 또는 완전히 다른 단어가 등장하는 것을 고려해서 해당 문자 시퀀스를 형태소라고 판단하는 단어 토크나이저라면 어떨까? <br>

예를 들어 에이비식스라는 문자열이 자주 연결되어 등장한다면 한 단어라고 판단하고, 또한 에이비식스라는 단어 앞뒤에 '최고', '가수', '실력'과 같은 독립된 다른 단어들이 계속해서 등장한다면 에이비식스를 한 단어로 파악하는 식이다. 그리고 이런 아이디어를 가진 단어 토크나이저가 soynlp이다.

## 2. 학습하기
soynlp는 기본적으로 학습에 기반한 토크나이저이므로 학습에 필요한 한국어 문서를 다운로드한다.

In [10]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [11]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x1b78b427ef0>)